In [1]:
#!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
#!unzip ml-20m.zip

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [3]:
import pandas as pd
# 훈련 데이터와 테스트 데이터를 나누기 위해 사용한다
from sklearn import model_selection

df = pd.read_csv("data/ml-20m/ratings.csv")
# X는 (userId, movieId) 쌍
X = df[["userId", "movieId"]].values
Y = df[["rating"]].values

# 훈련 데이터와 테스트 데이터를 9대 1로 분할
train_X, test_X, train_Y, test_Y\
    = model_selection.train_test_split(X, Y, test_size=0.1)

# X는 ID이고 정수이므로 int64, Y는 실수이므로 float32의 Tensor로 변환
train_dataset = TensorDataset(
    torch.tensor(train_X, dtype=torch.int64), torch.tensor(train_Y, dtype=torch.float32))
test_dataset = TensorDataset(
    torch.tensor(test_X, dtype=torch.int64), torch.tensor(test_Y, dtype=torch.float32))
train_loader = DataLoader(
    train_dataset, batch_size=1024, num_workers=3, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=3)


In [4]:
class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.item_emb = nn.Embedding(max_item, k, 0)
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        
        # user_feature*item_feature는 (batch_size,k) 차원이므로
        # k의 sum을 구하면 각 샘플의 내적이 된다
        out = torch.sum(user_feature * item_feature, 1)
        
        # [0, 5] 범위 내로 조정
        out = torch.sigmoid(out) * 5
        return out


In [5]:
max_user, max_item = X.max(0)
# np.int64형을 파이썬의 표준 int로 캐스트
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

In [6]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        ys.append(y)
        with torch.no_grad():
            ypred = net(x).to(device).view(-1)
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score.item()


In [7]:
from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)


100%|██████████| 17579/17579 [01:29<00:00, 197.17it/s]


0 1.6040153106132786 0.7372159957885742


100%|██████████| 17579/17579 [01:28<00:00, 198.10it/s]


1 0.887171385444752 0.7126760482788086


100%|██████████| 17579/17579 [01:28<00:00, 197.66it/s]


2 0.8397166714969908 0.7027606964111328


100%|██████████| 17579/17579 [01:28<00:00, 197.56it/s]


3 0.8152045769641014 0.6980058550834656


100%|██████████| 17579/17579 [01:30<00:00, 195.05it/s]


4 0.8017700441570316 0.6951594948768616


In [8]:
# 훈련한 모델을 CPU로 이동
net.to("cpu")

# 사용자1의 영화 10에 대한 평가 계산
query = (1, 10)

# int64 Tensor로 변환하고 batch 차원을 추가
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

# net에 전달
net(query)

tensor([3.6663], grad_fn=<MulBackward0>)

In [9]:
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1).float()
], 1).long()

# scores는 상위 k개의 점수
# indices는 상위 k개의 위치, 즉 movieId
scores, indices = torch.topk(net(query), 5)

In [10]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item,
                 user_k=10, item_k=10,
                 hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
            
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # 사용자 특이량과 상품 특이량을 모아서 하나의 벡터로 만듦
        out = torch.cat([user_feature, item_feature], 1)
        # 모은 벡터를 MLP에 넣는다
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()



In [11]:
import csv
from sklearn.feature_extraction.text import CountVectorizer

# csv.DictReader를 사용해서 CSV 파일 읽기
# 필요한 부분만 추출
with open("data/ml-20m/movies.csv") as fp:
    reader = csv.DictReader(fp)
    def parse(d):
        movieId = int(d["movieId"])
        genres = d["genres"]
        return movieId, genres
    data = [parse(d) for d in reader]

movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# 데이터에 맞추어 CountVectorizer를 훈련
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# key가 movieId이고 value가 BoW인 Tensor의 dict 만들기
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))


In [12]:
def first(xs):
    it = iter(xs)
    return next(it)

class MovieLensDataset(Dataset):
    def __init__(self, x, y, genres):
        assert len(x) == len(y)
        self.x = x
        self.y = y
        self.genres = genres
        
        # 장르 사전에 없는 movieId를 위한 더미 데이터
        self.null_genre = torch.zeros_like(
            first(genres.values()))
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # x = (userId, movieId)
        movieId = x[1]
        g = self.genres.get(movieId, self.null_genre)
        return x, y, g


In [13]:
train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32), 
    genre_dict)
test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict)
train_loader = DataLoader(
    train_dataset, batch_size=1024, shuffle=True, 
num_workers=3)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=3)

In [14]:
class NeuralMatrixFactorization2(nn.Module):
    def __init__(self, max_user, max_item, num_genres,
                 user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            # num_genres분만큼 차원이 늘어난다
            nn.Linear(user_k + item_k + num_genres, 
hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x, g):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # 장르 BoW를 cat로 특이 벡터에 결합한다
        out = torch.cat([user_feature, item_feature, g], 1)
        out = self.mlp(out)
        out = torch.sigmoid(out) * 5
        return out.squeeze()


In [15]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    # loader는 장르 BoW도 반환
    for x, y, g in loader:
        x = x.to(device)
        y = y.to(device)
        g = g.to(device)
        ys.append(y)
        # userId, movieId 외에 장르 BoW
        # 도 신경망 함수에 전달
        with torch.no_grad():
            ypred = net(x, g).to(device)
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

In [16]:
from statistics import mean

net = NeuralMatrixFactorization2(
    max_user+1, max_item+1, num_genres)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()
net.to("cuda:0")
for epoch in range(5):
    loss_log = []
    net.train()
    for x, y, g in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        g = g.to("cuda:0")
        o = net(x, g)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    net.eval()
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score.item(), flush=True)

100%|██████████| 17579/17579 [01:30<00:00, 194.82it/s]


0 0.7547865709256744 0.6500378251075745


100%|██████████| 17579/17579 [01:30<00:00, 195.17it/s]


1 0.6930769842004225 0.6331114768981934


100%|██████████| 17579/17579 [01:30<00:00, 194.59it/s]


2 0.6716549206772818 0.6307328939437866


100%|██████████| 17579/17579 [01:30<00:00, 195.21it/s]


3 0.6594974837501719 0.6300789713859558


100%|██████████| 17579/17579 [01:33<00:00, 188.65it/s]


4 0.6464216315039013 0.6229234933853149


In [17]:
# 지정한 위치만 1이고 나머지가 0인 Tensor 반환
def make_genre_vector(i, max_len):
    g = torch.zeros(max_len)
    g[i] = 1
    return g
  
query_genres = [make_genre_vector(i, num_genres)
    for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres분만큼 userId=100과 movieId=0의 Tensor를 만들어 결합
query = torch.stack([
    torch.empty(num_genres, dtype=torch.int64).fill_(100),
    torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

# GPU로 전송
query_genres = query_genres.to("cuda:0")
query = query.to("cuda:0")

# 점수 계산
net(query, query_genres)

tensor([3.2292, 3.2299, 3.2308, 3.2312, 3.2303, 3.2299, 3.2307, 3.2303, 3.2301,
        3.2309, 3.2317, 3.2322, 3.2316, 3.2315, 3.2322, 3.2306, 3.2311, 3.2330,
        3.2294, 3.2310, 3.2309, 3.2306, 3.2309, 3.2299], device='cuda:0',
       grad_fn=<SqueezeBackward0>)